UNIFICAR DISTINTOS MESES DE VENTAS MOSTRADOR


In [1]:
import sys
from pathlib import Path
import csv 

# Agregamos la carpeta src al path para poder importar
path_inicio = Path.cwd().parent/"src"
sys.path.append(str(path_inicio))

from main import unificar_ventas_excel_a_csv
from utilidades.constantes import DATA_PATH, PATRON_ARCHIVOS_PATH, ARCHIVO_SALIDA_PATH, DATA_FACTURAS_PATH, DATA_FINAL_FACTURAS_PATH

In [ ]:
unificar_ventas_excel_a_csv(
    DATA_PATH,
    PATRON_ARCHIVOS_PATH,
    ARCHIVO_SALIDA_PATH
)

UNIFICAR FACTURAS DE ARCA


In [2]:
import pandas as pd
from pathlib import Path
import re

def parse_line(line, file_name=None, line_num=None):
    try:
        if len(line) < 120:
            print(f"[{file_name} | línea {line_num}] Línea muy corta ({len(line)}): {line.strip()}")
            return None

        match = re.search(r"\d{15}", line[74:])
        if match:
            pos_importe = 74 + match.start()
        else:
            print(f"[{file_name} | línea {line_num}] No se encontró importe: {line.strip()}")
            return None

        nombre = line[74:pos_importe].strip()
        nombre = re.sub(r'^\d+', '', nombre)

        d = {
            "fecha_emision": line[0:8],
            "tipo_comprobante": line[8:11],
            "pto_venta": line[11:16],
            "nro_comprobante": line[16:36].lstrip("0"),
            "cuit_receptor": line[62:74].lstrip("0"),
            "denominacion": nombre,
            # Ahora el importe queda entero, sin decimales
            "importe_total": int(float(line[pos_importe:pos_importe+15]) / 100),
            "archivo_origen": file_name
        }
        return d
    except Exception as e:
        print(f"[{file_name} | línea {line_num}] ERROR: {e} - {line.strip()}")
        return None

DATA_PATH = Path("../data")
DATA_PROCESADA_PATH = Path("../data_procesada")
DATA_PROCESADA_PATH.mkdir(exist_ok=True)

ARCHIVO_SALIDA_PATH = DATA_PROCESADA_PATH / "ventas_final.csv"

data = []
for ventas_file in DATA_PATH.glob("VENTAS_*.txt"):
    with ventas_file.open(encoding="latin-1") as f:
        for i, line in enumerate(f, 1):
            parsed = parse_line(line, file_name=ventas_file.name, line_num=i)
            if parsed is not None:
                data.append(parsed)

df = pd.DataFrame(data)

if len(df) == 0:
    print("No se parseó ningún registro. Verificá el formato de los archivos.")
else:
    df["fecha_emision"] = pd.to_datetime(df["fecha_emision"], format="%Y%m%d", errors="coerce")
    df["fecha_emision"] = df["fecha_emision"].dt.strftime('%Y-%m-%d')
    df.to_csv(ARCHIVO_SALIDA_PATH, index=False, encoding="utf-8-sig")
    print(f"¡Listo! {len(df)} comprobantes unificados en {ARCHIVO_SALIDA_PATH}")
    print("Columnas exportadas:", df.columns.tolist())


¡Listo! 397 comprobantes unificados en ..\data_procesada\ventas_final.csv
Columnas exportadas: ['fecha_emision', 'tipo_comprobante', 'pto_venta', 'nro_comprobante', 'cuit_receptor', 'denominacion', 'importe_total', 'archivo_origen']
